In [82]:
import itertools

somelists = [
#    [1, 2],
#    [2, 3],
#    [1, 3]
    ['0', '0~'],
    ['1', '1~'],
    ['0', '0~']
]

lists = [
         [1, 2],
         [1, 3],
         [2, 3]
]

print(*itertools.product(*somelists))
print(*itertools.product(*lists))

#for element in itertools.product(*somelists):
#    print(element)

('0', '1', '0') ('0', '1', '0~') ('0', '1~', '0') ('0', '1~', '0~') ('0~', '1', '0') ('0~', '1', '0~') ('0~', '1~', '0') ('0~', '1~', '0~')
(1, 1, 2) (1, 1, 3) (1, 3, 2) (1, 3, 3) (2, 1, 2) (2, 1, 3) (2, 3, 2) (2, 3, 3)


In [83]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D, E, F, G, H


# to_cnf((A & B) | (C & D) | (E & F) | (G & H))
to_cnf((A & B) | (C & D) | (E & F))
# to_cnf((A & B & C) | (D & E & F))

(A | C | E) & (A | C | F) & (A | D | E) & (A | D | F) & (B | C | E) & (B | C | F) & (B | D | E) & (B | D | F)

In [3]:
to_cnf((~A & ~B) | (~C & ~D) | (~E & ~F))

(~A | ~C | ~E) & (~A | ~C | ~F) & (~A | ~D | ~E) & (~A | ~D | ~F) & (~B | ~C | ~E) & (~B | ~C | ~F) & (~B | ~D | ~E) & (~B | ~D | ~F)

In [46]:

to_cnf(((~A | B) & (~B | A)) & ((~C | D) & (~D | C)))

(A | ~B) & (B | ~A) & (C | ~D) & (D | ~C)

# Packages

In [4]:
%%capture
!pip install python-sat

In [5]:
from pysat.examples.fm import FM
from pysat.formula import WCNF
from itertools import combinations
import math

# Task

In [12]:
from pysat.formula import CNF
from pysat.formula import WCNF

In [7]:
n_neurons = 3
n_train = 3
n_features = n_neurons

assert(n_neurons % 2 == 1)  # odd number of neurons is allowed

def check_if_2nd_feature_true(X):
    return [x_i[1] is True for x_i in X]

def prepare_dataset(n_train_, n_features_, func_):
    X = [
         [True, False, False],
         [False, True, False],
         [True, True, False]
    ]
    Y = func_(X)
    return X, Y

X, Y = prepare_dataset(
    n_train_ = n_train, 
    n_features_ = n_features,
    func_ = check_if_2nd_feature_true
)

In [8]:
print(X)
print(Y)

[[True, False, False], [False, True, False], [True, True, False]]
[False, True, True]


In [22]:
# wcnf = WCNF()

# wcnf.append([-1, 2]) 
# wcnf.append([-2, 3])
# wcnf.append([-3, True])
# # wcnf.append([-4, False])

# print(wcnf.soft)
# print(wcnf.hard)
# print(wcnf.wght)

[]
[[-1, 2], [-2, 3], [-3, True]]
[]


In [23]:
# fm = FM(wcnf,verbose=10)
# fm.compute()

True

In [37]:

print(int(math.ceil(4.2)))

5


In [25]:
# Encoding weights
propositional_weights = [i for i in range(n_neurons)]
print(propositional_weights)

[0, 1, 2]


In [74]:
combos = combinations(propositional_weights, group_size)
# print(*combos)
l = list(combos)
print(l)

[(0, 1), (0, 2), (1, 2)]


In [77]:
# Weighted CNF form
wcnf = WCNF()

group_size = int(math.ceil(n_neurons / 2))
print(f'Group size (for binomial coefficients = {group_size})')

for x, y in zip(X, Y):
    print(f'Encoding x = {x}, y = {y}')
    combos = list(combinations(propositional_weights, group_size))
    print(combos)

    cartesian_prod = itertools.product(*combos)
    print(*cartesian_prod)

    # for w_combination in combinations(propositional_weights, group_size):
    #     # print(c, c[0], c[1])
    #     # print(propositional_weights.index(c[0]), propositional_weights.index(c[1]))
    #     print(f'weight combination = {w_combination}')
        
    #     for weight in w_combination:
    #         # index of weight --> the same as index of feature of current object x_i in the X dataset
    #         w_index = propositional_weights.index(weight)
    #         x_feature = x[w_index]
    #         print(f'w_{w_index}, x_j = {x_feature}')

    #         # Our clause is - weight == x_feature (equivalence)
    #         # which is (not(weight) V x_feature) and (not(x_feature) V weight)
            
    #         first_clause = [-weight, x_feature]
    #         second_clause = [weight, -x_feature]
    #         wcnf.append



Group size (for binomial coefficients = 2)
Encoding x = [True, False, False], y = False
[(0, 1), (0, 2), (1, 2)]
(0, 0, 1) (0, 0, 2) (0, 2, 1) (0, 2, 2) (1, 0, 1) (1, 0, 2) (1, 2, 1) (1, 2, 2)
Encoding x = [False, True, False], y = True
[(0, 1), (0, 2), (1, 2)]
(0, 0, 1) (0, 0, 2) (0, 2, 1) (0, 2, 2) (1, 0, 1) (1, 0, 2) (1, 2, 1) (1, 2, 2)
Encoding x = [True, True, False], y = True
[(0, 1), (0, 2), (1, 2)]
(0, 0, 1) (0, 0, 2) (0, 2, 1) (0, 2, 2) (1, 0, 1) (1, 0, 2) (1, 2, 1) (1, 2, 2)


In [27]:
# No hard clauses are introduced.
# Soft clauses - have weight 1, if the prediction of the network is correct
# for a given instance (x_i, y_i) in a dataset

In [28]:
# Encode this into clauses
for x_i, y_i in zip(X, Y):
    print(f'Encoding x_i = {x_i}, y_i = {y_i}')
    

Encoding x_i = [True, False, False]
Encoding y_i = False
Encoding x_i = [False, True, False]
Encoding y_i = True
Encoding x_i = [True, True, False]
Encoding y_i = True
